# Arabica Coffee Exploration

## Introduction

In the past few years, I have become more interested in coffee. I have thought about how Q-grade affect the price of coffee. However, a more foundational questions deals with how a coffee will score first before it is priced. A Q-grader grades the the coffee during a process called cupping. The cupping process allows the grader to compare different coffees together. They brew each coffee with the same amount of coffee and water. Specialty Coffee Assocation (SCA) provides the [protocols](https://sca.coffee/research/protocols-best-practices) for cupping an evaluating coffee. 

## Data
This data comes from James LeDoux [GitHub](https://github.com/jldbc/coffee-quality-database/tree/master/data) repositiary. He scrapped the data from Coffee Quality Institue [database](https://database.coffeeinstitute.org/) in January 2018. He has both the scrapped data and the clean data. I have choosen to use the clean data.

## Scoping

## Import Python Modules

## Loading the Data